In [1]:
# All Imports
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from amazon.api import AmazonAPI

%matplotlib inline

In [2]:
def get_amazon(config_file):
    """ Read the config_file and construct an instance of AmazonAPI.
    Args:
      config_file ... A config file in ConfigParser format with Amazon credentials
    Returns:
      An instance of AmazonAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    amazon = AmazonAPI(
                   config.get('amazon', 'AMAZON_ACCESS_KEY'),
                   config.get('amazon', 'AMAZON_SECRET_KEY'),
                   config.get('amazon', 'AMAZON_ASSOC_TAG'))
    return amazon

amazon = get_amazon('amazon.cfg')
print('Established Amazon connection.')

Established Amazon connection.


In [4]:
from lxml.etree import fromstring
import urllib

product = amazon.lookup(ItemId='B00EOE0WKQ')
help(product)
url =product.reviews[1]
print url
data = urllib.urlopen(url) # fetching url with iframe
#print data.read()
#print fromstring(data.read())
"""
tree = fromstring(data.read()) # parsing tree to get src attribute
src_url = tree.cssselect("iframe").attrib['src']
data = urllib.urlopen(src_url) # open iframe src url
tree = fromstring(data.read())
"""

Help on AmazonProduct in module amazon.api object:

class AmazonProduct(LXMLWrapper)
 |  A wrapper class for an Amazon product.
 |  
 |  Method resolution order:
 |      AmazonProduct
 |      LXMLWrapper
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, item, aws_associate_tag, api, *args, **kwargs)
 |      Initialize an Amazon Product Proxy.
 |      
 |      :param item:
 |          Lxml Item element.
 |  
 |  get_attribute(self, name)
 |      Get Attribute
 |      
 |      Get an attribute (child elements of 'ItemAttributes') value.
 |      
 |      :param name:
 |          Attribute name (string)
 |      :return:
 |          Attribute value (string) or None if not found.
 |  
 |  get_attribute_details(self, name)
 |      Get Attribute Details
 |      
 |      Gets XML attributes of the product attribute. These usually contain
 |      details about the product attributes such as units.
 |      :param name:
 |          Attribute name (string)
 |      :r

'\ntree = fromstring(data.read()) # parsing tree to get src attribute\nsrc_url = tree.cssselect("iframe").attrib[\'src\']\ndata = urllib.urlopen(src_url) # open iframe src url\ntree = fromstring(data.read())\n'